# Imports & Constants

<font size="4">Imports </font>

In [54]:
import os
import torch.nn as nn
import zipfile
import requests
from glob import glob

<font size="4">Constants</font>

In [7]:
current_working_directory = os.getcwd()
DATA_BASE_DIRECTORY: str = os.path.join(current_working_directory, 'data')
TRAINING_SET_URL='https://web.archive.org/web/20241214060505/https://vis-www.cs.umass.edu/lfw/pairsDevTrain.txt'
TEST_SET_URL='https://web.archive.org/web/20241214070147/https://vis-www.cs.umass.edu/lfw/pairsDevTest.txt#expand'

# Acquiring & Handling Data

<font size="6">Acquiring_Data</font>

In [45]:
!pip install -q gdown

def download_images_from_drive(file_id: str, zip_path: str) -> str:
  """
  Downloads images from drive and return the path to the extracted folder, but 1
  level down assuming the structure of the directories are known in advance.
  """
  file_location: str = os.path.join(DATA_BASE_DIRECTORY, 'lfw2', 'lfw2')
  if os.path.exists(file_location):
    print(f"Dataset already downloaded to {file_location}")
    return file_location
  !gdown {file_id} -O {zip_path}

  os.makedirs(DATA_BASE_DIRECTORY, exist_ok=True)
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(DATA_BASE_DIRECTORY)
  !rm {zip_path}
  print(f"Dataset extracted to {DATA_BASE_DIRECTORY}")
  return file_location

updated_dir_location: str = download_images_from_drive(
    file_id="1p1wjaqpTh_5RHfJu4vUh8JJCdKwYMHCp",
    zip_path="dataset.zip"
    )

Dataset already downloaded to /content/data/lfw2/lfw2
/content/data/lfw2/lfw2


<font size="4">Loading file paths to memory</font>

In [51]:
def loads_files_paths_to_memory(base_directory: str, image_format: str = '.jpg') -> None:
    images: dict[str, dict[str, str]] = dict()
    images_loaded: int = 0
    for root, subdirs, files in os.walk(base_directory):
        if root == base_directory:
            continue
        person_name: str = root.split(os.sep)[-1]
        if person_name not in images:
            images[person_name] = dict()
        for file in files:
            if not file.endswith(image_format):
                raise Warning(f"File {file} is not a {image_format} file. Continuing...")
                continue
            stripped_image: str = file.rstrip(image_format) # File without ending
            image_index: int = int(stripped_image.split('_')[-1])
            if image_index in images[person_name]:
                 raise ValueError(f"Index: {image_index} collision for: {person_name}")
            images[person_name][image_index] = os.path.join(root, file)
            images_loaded += 1
    if len(images) < 1:
        raise ValueError(f"No images were found in {base_directory}, aborting...")
    print(f"People scanned: {len(images)}")
    print(f"Images loaded: {images_loaded}")
    return images

<font size="4">Organizing According to example-validation-train</font>

In [ ]:
def parse_train_test_txt(url_to_use: str):
    url_response = requests.get(url_to_use)
    if url_response.status_code == 200:
        text_content = url_response.text
    else:
        raise ValueError("Invalid URL")
    ret_text: list[str] = text_content.split('\n')
    positive_examples: list[tuple[str, int, int]] = list()
    negative_examples: list[tuple[tuple[str, int], tuple[str, int]]] = list()
    for text in ret_text:
        separated_by_tabs: list[str] = text.split('\t')
        if len(separated_by_tabs) < 3:
            # This is the number in the beginning
            continue
        if len(separated_by_tabs) == 3:
            # This is a positive example (2 Pictures of the same person)
            person = separated_by_tabs[0]
            first_image_index = int(separated_by_tabs[1])
            second_image_index = int(separated_by_tabs[2])
            positive_examples.append((person, first_image_index, second_image_index))
        if len(separated_by_tabs) == 4:
            first_person = separated_by_tabs[0]
            first_person_image_index = int(separated_by_tabs[1])
            second_person = separated_by_tabs[2]
            second_person_image_index = int(separated_by_tabs[3])
            negative_examples.append(
                                        (
                                             (first_person, first_person_image_index),
                                             (second_person, second_person_image_index)
                                        )
                                     )
    return positive_examples, negative_examples

  training_positive_examples, training_negative_examples = parse_train_test_txt(TRAINING_SET_URL)
  test_positive_examples, test_negative_examples = parse_train_test_txt(TEST_SET_URL)

<font size="6">Handling_Data</font>

<font size="6">Creating Network</font>

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=10),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),  # TODO Change the in channels to support more colors.
            nn.ReLU(),
        )

        self.fc1 = nn.Sequential(
            nn.Flatten(),                         # -> [256*6*6 = 9216]
            nn.Linear(256*6*6, 4096),
            nn.Sigmoid()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1),
            nn.Sigmoid()  # similarity score
        )

    def forward_once(self, x):
        x = self.cnn(x)
        x = self.fc1(x)
        return x

    def forward(self, input1, input2):
        out1 = self.forward_once(input1)
        out2 = self.forward_once(input2)
        # L1 distance
        diff = torch.abs(out1 - out2)
        similarity_score = self.fc2(diff)
        return similarity_score



# asd = ConvNet()